In [1]:
import diagonal_b6 as b6
import json

w = b6.connect_insecure("0.0.0.0:8002")

### Run b6

```
run-b6 --world ../../data/indexes/edinburgh-241113.connected.index
```

### Prep data

In [2]:
with open("cpc_demo.geojson", "r") as f:
    geojson = json.load(f)

for f in geojson["features"]:
    f["properties"]["#potential_green"] = "yes"
    f["properties"]["id"] = str(f["properties"]["id"])
    
    if f["properties"]["#type"]== "wide corner":
        f["properties"]["b6:colour"] = "#FF8465"
        
    if f["properties"]["#type"]=="road carriageway fragment":
        f["properties"]["b6:colour"] = "#f40ea8"

    f["properties"]["#building"] = "yes"

### Import and run

In [3]:
name = "diagonal.works/cpc"

castle        = b6.osm_node_id(26005891)
root          = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 500)
collection_id = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 501)

add_features   = b6.import_geojson(b6.parse_geojson(json.dumps(geojson)), name)

# TODO: Bug; why does this need to be ran here?
w(add_features)

add_collection = b6.add_collection(collection_id, {}, b6.find(b6.keyed("#potential_green")))

add_root = b6.add_collection(root, {}, b6.collection(
    b6.pair("centroid", b6.ll(55.939231, -3.208273)),
    b6.pair("docked", collection_id),
))

changes = b6.merge_changes([ 
    add_root, 
    add_features,
    add_collection,
])

w(b6.add_world_with_change(root, changes))

print(f"http://localhost:8001/?r={root}")

http://localhost:8001/?r=/collection/diagonal.works/cpc/500
